In [5]:
class Cycliste:
    from Logger      import Logger
    nb_d_instance =0
    cycliste_logger= Logger("./cyclistes.csv")
    def __init__(self, position_home, position_travail, ville):
        import random
        import numpy as np
        from Logger      import Logger
        from Trajet      import Trajet

        Cycliste.nb_d_instance +=1
        self.id           = Cycliste.nb_d_instance
        self.home         = position_home
        self.travail      = position_travail
        self.position     = position_home
        self.destination  = position_travail
        self.sur_velo     = False
        self.velo         = False
        self.sexe         = random.choice(["H", "F"])
        self.age          = random.choice(range(15,85))
        self.sportivite   = max(np.random.normal(2, 2), 0.1)
        self.vitesse_a_pied =  2 * self.sportivite / 10
        self.vitesse_a_velo = 5 * self.sportivite  / 10
        self.ville        = ville
        self.trajet       = Trajet(self.home, self.home)
        self.positions    = [self.position]
        self.timestamps   = [0]
        self.destination_finale = False
        self.vitesse      = self.vitesse_a_pied
        self.wanted_details = [ 'id', 'vitesse_a_pied', 'vitesse_a_velo', 'trajet', 'sur_velo', 'velo','vitesse', 'trajet', 'destination_finale']
        self.status       = [{x : self.__dict__[x] for x in self.wanted_details}]
        self.velo_perf_minimale = 0.4
    def __str__(self):
        return " %s :  %s" %(self.id,  str(self.position))
    
    def get_historique(self):
        import pandas as pd
        df = pd.concat([x.get_df() for x in self.positions])
        df["id_cycliste"] = self.id
        df["time"]        = self.timestamps
        df["status"]      = self.status
        
        return df
    
    def get_vitesse(self):
        if self.velo:
            self.vitesse = self.vitesse_a_velo * self.velo.performance
        else:
            self.vitesse =  self.vitesse_a_pied
        return self.vitesse
        
    def est_a_la_maison(self):
        return self.position == self.home
    
    def est_au_travail(self):
        return self.position == self.travail
    
    def est_arrive(self):
        return self.position == self.destination_finale
    
    def log(self, message):
        message="cycliste_%s : %s"%(self.id, message)
        Cycliste.cycliste_logger.log(message)
    def est_sur_station(self):
        return self.ville.prestataire.is_position_a_station(self.position)

    def ou_suis_je(self):
        if self.est_a_la_maison():
            return " a la maison"
        if self.est_au_travail():
            return " au travail"

        return " en route"
        
    def ou_vais_je(self):
        if self.destination == self.travail:
            return " au travail"
        if self.destination == self.home:
            return " a la maison"
        

    def avancer(self, timestamp, debug=False):
        from Trajet import Trajet
        if not self.trajet.fini():
            self.position = self.trajet.next_position()
        else:            
            if self.est_a_la_maison():
                self.destination        = self.ville.prestataire.get_closest_station(self.position).position
                self.destination_finale = self.travail
                self.sur_velo           = False
                self.trajet             = Trajet(self.home, self.destination, self.get_vitesse())
                self.position           = self.home

            if self.est_au_travail():
                self.destination        = self.ville.prestataire.get_closest_station(self.position).position
                self.destination_finale = self.home
                self.sur_velo           = False
                self.trajet             = Trajet(self.travail, self.destination, self.get_vitesse())
                self.position           = self.travail
                
            if self.est_sur_station():
                station_actuelle = self.ville.prestataire.get_station_at_position(self.position)
                self.position    = station_actuelle.position
                if self.sur_velo:
                    velo_rendu       = station_actuelle.deposer_un_velo(self.velo)
                    if velo_rendu:
                        self.sur_velo    = False
                        self.velo        = False
                        self.destination = self.destination_finale
                        
                    else:
                        self.destination = self.ville.prestataire.get_closest_station(self.position).position
                else:
                    velo_pris = station_actuelle.prendre_un_velo()
                    if velo_pris:
                        if self.velo_perf_minimale <= velo_pris.performance:
                            self.sur_velo    = True
                            self.velo        = velo_pris
                            self.destination = self.ville.prestataire.get_closest_station(self.destination_finale).position
                        else:
                            new_velo     = velo_pris
                            velos_testes = [new_velo]
                            while new_velo.performance <self.velo_perf_minimale and station_actuelle.has_velo():
                                new_velo = station_actuelle.prendre_un_velo()
                                velos_testes.append (new_velo)
                            if new_velo.performance <self.velo_perf_minimale:
                                [station_actuelle.deposer_un_velo(velo) for velo in velos_testes]
                                self.destination = self.ville.prestataire.get_closest_station(self.position).position 
                            else:
                                self.sur_velo    = True
                                self.velo        = velos_testes.pop()
                                [station_actuelle.deposer_un_velo(velo) for velo in velos_testes]
                                self.destination = self.ville.prestataire.get_closest_station(self.destination_finale).position
                             
                            
                    else:
                        self.destination = self.ville.prestataire.get_closest_station(self.position).position 
                self.trajet      = Trajet(self.position, self.destination, self.get_vitesse())
        self.positions.append(self.position)
        self.timestamps.append(timestamp)
        self.status.append({x : str(self.__dict__[x]) for x in self.wanted_details})

         

importing Jupyter notebook from Logger.ipynb
